### Распознавание лиц

In [1]:
import numpy as np
import cv2
import os
from sklearn.neighbors import KNeighborsClassifier

In [25]:
!curl -o haarcascade_frontalface_alt.xml  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  660k  100  660k    0     0   660k      0  0:00:01 --:--:--  0:00:01 1280k


In [26]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

In [27]:
skip = 0
face_data = []
dataset_path = './data/'
os.makedirs(dataset_path, exist_ok=True)

In [28]:
file_name = input("Enter the name of the person : ")
while True:
    ret,frame = cap.read()

    if ret==False:
        continue

    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if len(faces)==0:
        continue

    faces = sorted(faces,key=lambda f:f[2]*f[3])

    # Pick the last face (because it is the largest face acc to area(f[2]*f[3]))
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

        #Extract (Crop out the required face) : Region of Interest
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        skip += 1
        if skip%10==0:
            face_data.append(face_section)
            print(len(face_data))


    cv2.imshow("Frame",frame)
    cv2.imshow("Face Section",face_section)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

np.save(os.path.join(dataset_path, file_name+'.npy'),face_data)
print("Data Successfully save at "+dataset_path+file_name+'.npy')

cap.release()
cv2.destroyAllWindows()

Enter the name of the person : Kotya
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
(25, 30000)
Data Successfully save at ./data/Kotya.npy


In [2]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

In [3]:
skip = 0
dataset_path = './data/'

In [4]:
face_data = [] 
labels = []

In [5]:
class_id = 0
names = {}

In [6]:
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)

        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

Loaded Ivan.npy
Loaded Kotya.npy
Loaded Putin.npy
Loaded Trump.npy


In [7]:
face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0)

In [8]:
print(face_dataset.shape)
print(face_labels.shape)

(105, 30000)
(105,)


In [9]:
face_dataset

array([[ 39,  56,  83, ...,  26,  15,  14],
       [ 46,  20,  16, ...,  41,  17,  14],
       [ 45,  52,  85, ...,  34,  19,  15],
       ...,
       [124, 114, 124, ..., 225, 183, 168],
       [130, 117, 127, ..., 220, 173, 160],
       [134, 128, 132, ..., 183, 144, 130]], dtype=uint8)

In [10]:
clf = KNeighborsClassifier(n_neighbors=5)

In [11]:
clf.fit(face_dataset, face_labels)

KNeighborsClassifier()

In [12]:
while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        out = clf.predict(face_section.reshape(1,-1))
        dist = clf.kneighbors(face_section.reshape(1,-1))[0][0][0]

        pred_name = f'{names[int(out)]} {dist:.0f}'
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


### Семантический поиск

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

https://www.kaggle.com/devendra45/movies-similarity

In [2]:
df_movies = pd.read_csv('movies.csv')
df_movies.head()

,rank,title,genre,wiki_plot,imdb_plot
0,0,The Godfather,"[u' Crime', u' Drama']","On the day of his only daughter's wedding, Vit...","In late summer 1945, guests are gathered for t..."
1,1,The Shawshank Redemption,"[u' Crime', u' Drama']","In 1947, banker Andy Dufresne is convicted of ...","In 1947, Andy Dufresne (Tim Robbins), a banker..."
2,2,Schindler's List,"[u' Biography', u' Drama', u' History']","In 1939, the Germans move Polish Jews into the...",The relocation of Polish Jews from surrounding...
3,3,Raging Bull,"[u' Biography', u' Drama', u' Sport']","In a brief scene in 1964, an aging, overweight...","The film opens in 1964, where an older and fat..."
4,4,Casablanca,"[u' Drama', u' Romance', u' War']",It is early December 1941. American expatriate...,"In the early years of World War II, December 1..."


In [3]:
df_movies.shape

(100, 5)

$ {\displaystyle \mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}} $

$ {\displaystyle \mathrm {idf} (t,D)=\log {\frac {|D|}{|\{\,d_{i}\in D\mid t\in d_{i}\,\}|}}} $

|D| — число документов в коллекции;  
$ {\displaystyle |\{\,d_{i}\in D\mid t\in d_{i}\,\}|} $ — число документов из коллекции $ {\displaystyle D}D $, в которых встречается $ {\displaystyle t}t $ (когда $ {\displaystyle n_{t}\neq 0}{\displaystyle n_{t}\neq 0} $).

In [4]:
vect = TfidfVectorizer()

In [5]:
nn = NearestNeighbors(n_neighbors=5)

In [6]:
nn.fit(vect.fit_transform(df_movies['wiki_plot']))

NearestNeighbors()

In [22]:
def get_top_films(text):
    vec = vect.transform([text])
    display(vec.todense().shape)
    
    dist, neigh = nn.kneighbors(vec)
    print(dist)
    print(neigh)
    res = df_movies.loc[neigh[0]]
    return res[['title', 'genre']]

In [23]:
get_top_films('crime and drugs')

(1, 10599)

[[1.32350634 1.35445313 1.36207106 1.37661088 1.37730399]]
[[59 63 91 80 15]]


,title,genre
59,Goodfellas,"[u' Biography', u' Crime', u' Drama']"
63,The French Connection,"[u' Action', u' Crime', u' Thriller']"
91,A Clockwork Orange,"[u' Crime', u' Drama', u' Sci-Fi']"
80,The Green Mile,"[u' Crime', u' Drama', u' Fantasy', u' Mystery']"
15,On the Waterfront,"[u' Crime', u' Drama']"


In [11]:
get_top_films('violence protest march')

[[1.36452096 1.38213937 1.39334214 1.3997055  1.4059172 ]]
[[32 91 30 44  8]]


,title,genre
32,Gandhi,"[u' Biography', u' Drama', u' History']"
91,A Clockwork Orange,"[u' Crime', u' Drama', u' Sci-Fi']"
30,Amadeus,"[u' Biography', u' Drama', u' Music']"
44,The Best Years of Our Lives,"[u' Drama', u' Romance', u' War']"
8,The Wizard of Oz,"[u' Adventure', u' Family', u' Fantasy', u' Mu..."


In [12]:
get_top_films('love affair hate')

[[1.38234655 1.38356728 1.39019241 1.3985436  1.3997051 ]]
[[74 84 18  3 71]]


,title,genre
74,Terms of Endearment,"[u' Comedy', u' Drama']"
84,The Graduate,"[u' Comedy', u' Drama', u' Romance']"
18,West Side Story,"[u' Crime', u' Drama', u' Musical', u' Romance..."
3,Raging Bull,"[u' Biography', u' Drama', u' Sport']"
71,Annie Hall,"[u' Comedy', u' Drama', u' Romance']"


In [13]:
get_top_films('fight action')

[[1.36668671 1.37361813 1.37927179 1.38699596 1.39003004]]
[[39 15 36  3 18]]


,title,genre
39,Rocky,"[u' Drama', u' Sport']"
15,On the Waterfront,"[u' Crime', u' Drama']"
36,Saving Private Ryan,"[u' Action', u' Drama', u' War']"
3,Raging Bull,"[u' Biography', u' Drama', u' Sport']"
18,West Side Story,"[u' Crime', u' Drama', u' Musical', u' Romance..."
